In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)



In [ ]:
# 📥 IHC_Cumulative_Ranking から直近7日分のデータ取得
spreadsheet = gc.open("IHC_Cumulative_Ranking")
sheets = sorted([ws.title for ws in spreadsheet.worksheets() if ws.title.startswith("Ranking-")])
latest_7_titles = sheets[-7:]

dfs = []
for title in latest_7_titles:
    df = get_as_dataframe(spreadsheet.worksheet(title)).dropna(how='all')
    df = df.set_index("Spotify ID")
    dfs.append(df)


In [ ]:
# 🔢 週間スコア集計
weekly_score = pd.DataFrame()
for df in dfs:
    if '今回獲得スコア' in df.columns:
        if weekly_score.empty:
            weekly_score = df[['今回獲得スコア']].copy()
        else:
            weekly_score = weekly_score.add(df[['今回獲得スコア']], fill_value=0)

weekly_score = weekly_score.rename(columns={'今回獲得スコア': 'total_score'})
weekly_score = weekly_score.join(dfs[-1][['集計時人気度']], how='left')
weekly_score = weekly_score.reset_index()

# アーティスト名を結合（Spotify IDをキーに）
dfs[-1] = dfs[-1].reset_index()
weekly_score = weekly_score.merge(dfs[-1][['Spotify ID', 'アーティスト名']], on='Spotify ID', how='left')
weekly_score = weekly_score[['Spotify ID', 'total_score', 'アーティスト名', '集計時人気度']]
weekly_score = weekly_score.sort_values(by='total_score', ascending=False)
weekly_score.insert(0, '順位', range(1, len(weekly_score) + 1))

# --- デバッグコードと前回順位取得のtryブロック ---
try:
    weekly_ranking_spreadsheet = gc.open("IHC_Weekly_Ranking")
    all_weekly_sheets = weekly_ranking_spreadsheet.worksheets()
    if all_weekly_sheets:
        latest_weekly_ranking_sheet_title = all_weekly_sheets[-1].title
        print(f"✅ デバッグ: IHC_Weekly_Rankingの最新ワークシートタイトルは '{latest_weekly_ranking_sheet_title}' です。")
        last_week_df = get_as_dataframe(weekly_ranking_spreadsheet.worksheet(latest_weekly_ranking_sheet_title)).dropna(how='all')
        last_week_df = last_week_df[['順位', 'Spotify ID']].rename(columns={'順位': '前回順位'})
        last_week_df = last_week_df.set_index('Spotify ID')

        # weekly_scoreに前回順位を結合
        weekly_score = weekly_score.set_index('Spotify ID')
        weekly_score = weekly_score.join(last_week_df, how='left')
        weekly_score = weekly_score.reset_index()
    else:
        print("⚠️ IHC_Weekly_Rankingにワークシートがないため、前回順位はスキップします。")
        weekly_score['前回順位'] = None
except gspread.SpreadsheetNotFound:
    print("⚠️ IHC_Weekly_Rankingスプレッドシートが見つからないため、前回順位はスキップします。")
    weekly_score['前回順位'] = None

# 順位と前回順位の並び替え
weekly_score = weekly_score[['順位', '前回順位', 'total_score', 'アーティスト名', '集計時人気度', 'Spotify ID']]

✅ デバッグ: IHC_Weekly_Rankingの最新ワークシートタイトルは '2026-0106to0112' です。


In [ ]:
# 📝 出力処理
output_title = f"{datetime.now().year}-{(datetime.now() - timedelta(days=6)).strftime('%m%d')}to{datetime.now().strftime('%m%d')}"

# スプレッドシートがなければ作成
try:
    output_spreadsheet = gc.open("IHC_Weekly_Ranking")
except gspread.SpreadsheetNotFound:
    output_spreadsheet = gc.create("IHC_Weekly_Ranking")
    print("✅ 新規スプレッドシート『IHC_Weekly_Ranking』を作成しました")

# ワークシート追加
try:
    output_spreadsheet.add_worksheet(title=output_title, rows="200", cols="20")
except:
    print("⚠️ 同名のワークシートが既に存在します。上書きします。")
worksheet = output_spreadsheet.worksheet(output_title)
worksheet.clear()
set_with_dataframe(worksheet, weekly_score[['順位', '前回順位', 'total_score', 'アーティスト名', '集計時人気度', 'Spotify ID']])

# URL出力
sheet_url = f"https://docs.google.com/spreadsheets/d/{output_spreadsheet.id}"
print("✅ 生成されたスプレッドシートのURL：", sheet_url)

✅ 生成されたスプレッドシートのURL： https://docs.google.com/spreadsheets/d/1ZNZ0QkimE5YL7argwTL6QKICG1dIB_mPtNmMWdj-lOo
